In [1]:
import tensorflow_hub as hub

2023-11-10 11:08:27.270982: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-10 11:08:27.398122: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-10 11:08:28.183144: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-11-10 11:08:28.183221: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [2]:
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds

/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import tensorflow_datasets as tfds

# Load IMDb reviews dataset
train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

# Convert TensorFlow Datasets tensors to NumPy arrays
train_examples, train_labels = tfds.as_numpy(train_data)
test_examples, test_labels = tfds.as_numpy(test_data)



In [6]:
import re
import string

def clean_text(text):
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    # Lowercase
    text = text.lower()
    # Remove special characters and punctuation
    text = re.sub(f'[{re.escape(string.punctuation)}]', '', text)
    return text

# Apply cleaning to your data
train_examples = [clean_text(text.decode('utf-8')) for text in train_examples]
test_examples = [clean_text(text.decode('utf-8')) for text in test_examples]


In [13]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_examples)

train_sequences = tokenizer.texts_to_sequences(train_examples)
test_sequences = tokenizer.texts_to_sequences(test_examples)

# Padding
max_len = max(len(seq) for seq in train_sequences)
train_padded = pad_sequences(train_sequences, maxlen=max_len, padding='post')
test_padded = pad_sequences(test_sequences, maxlen=max_len, padding='post')

# Convert sequences to strings
train_strings = [' '.join(map(str, seq)) for seq in train_padded]
test_strings = [' '.join(map(str, seq)) for seq in test_padded]


In [14]:
import tensorflow_hub as hub
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split


# Split the data into training and validation sets
train_strings, val_strings, train_labels, val_labels = train_test_split(train_strings, train_labels, test_size=0.2, random_state=42)

# Build the Universal Sentence Encoder embedding layer
embed_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4", input_shape=[], dtype=tf.string, trainable=False)




In [16]:
#Build the sentiment analysis model with additional LSTM layers
model = models.Sequential([
    embed_layer,
    layers.Reshape((1, -1)),  # Reshape the 2D tensor to 3D tensor
    layers.Bidirectional(layers.LSTM(64, return_sequences=True)),
    layers.Bidirectional(layers.LSTM(64)),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])



In [19]:
import numpy as np

# Convert strings to NumPy arrays
train_strings_np = np.array(train_strings)
val_strings_np = np.array(val_strings)

# Convert labels to NumPy arrays
train_labels_np = np.array(train_labels)
val_labels_np = np.array(val_labels)

# Train the model
model.fit(train_strings_np, train_labels_np, epochs=5, batch_size=32, validation_data=(val_strings_np, val_labels_np))


Epoch 1/10
625/625 [==============================] - 86s 126ms/step - loss: 0.6936 - accuracy: 0.4998 - val_loss: 0.6939 - val_accuracy: 0.4848
Epoch 2/10
625/625 [==============================] - 77s 124ms/step - loss: 0.6936 - accuracy: 0.4985 - val_loss: 0.6928 - val_accuracy: 0.5152
Epoch 3/10
625/625 [==============================] - 77s 124ms/step - loss: 0.6933 - accuracy: 0.4983 - val_loss: 0.6937 - val_accuracy: 0.4848
Epoch 4/10
625/625 [==============================] - 77s 124ms/step - loss: 0.6931 - accuracy: 0.5099 - val_loss: 0.6934 - val_accuracy: 0.4848
Epoch 5/10
625/625 [==============================] - 77s 124ms/step - loss: 0.6931 - accuracy: 0.5085 - val_loss: 0.6923 - val_accuracy: 0.5358
Epoch 6/10
625/625 [==============================] - 77s 124ms/step - los

In [24]:
test_loss, test_acc = model.evaluate(test_examples, test_labels)
print(f"Test accuracy: {test_acc}")


782/782 [==============================] - 20s 25ms/step - loss: 0.3226 - accuracy: 0.8582
Test accuracy: 0.8582000136375427


In [26]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
from tensorflow.keras import layers, models
import numpy as np

# Load IMDb reviews dataset
train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

train_examples, train_labels = tfds.as_numpy(train_data)
test_examples, test_labels = tfds.as_numpy(test_data)

# Convert strings to NumPy arrays
train_strings = np.array(train_examples)
test_strings = np.array(test_examples)

# Convert labels to NumPy arrays
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

# Load Universal Sentence Encoder from TensorFlow Hub
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
embed_layer = hub.KerasLayer(module_url, input_shape=[], dtype=tf.string, trainable=True)



# Build the sentiment analysis model with additional LSTM layers
model = models.Sequential([
    embed_layer,
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_strings, train_labels, epochs=1, batch_size=32, validation_split=0.1)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_strings, test_labels)
print(f"Test accuracy: {test_accuracy}")


 54/782 [=>............................] - ETA: 18s - loss: 0.2596 - accuracy: 0.9010

In [25]:
test_loss, test_acc = model.evaluate(test_examples, test_labels)
print(f"Test accuracy: {test_acc}")


782/782 [==============================] - 55s 70ms/step - loss: 0.2772 - accuracy: 0.8804
Test accuracy: 0.8804000020027161
